In [1]:
# Tratamiento de datos
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV

# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

import datetime

In [2]:
df = pd.read_pickle("datos/4.casual_encoding_oh_final.pkl")
df.head()

,yr,mnth,dia_anual,holiday_num,weekday_num,workingday_num,weathersit,atemp,casual,cnt,seasons_autumn,seasons_spring,seasons_summer,seasons_winter
0,0,1,1,1,0,0,2,18.18125,331,985,0.0,0.0,0.0,1.0
1,0,1,2,0,1,1,2,17.68695,131,801,0.0,0.0,0.0,1.0
2,0,1,3,0,2,1,1,9.47025,120,1349,0.0,0.0,0.0,1.0
3,0,1,4,0,3,1,1,10.60610,108,1562,0.0,0.0,0.0,1.0
4,0,1,5,0,4,1,1,11.46350,82,1600,0.0,0.0,0.0,1.0


In [3]:
#df.drop(["cnt"], axis=1, inplace=True)

In [4]:
df.dtypes

yr                category
mnth              category
dia_anual         category
holiday_num       category
weekday_num       category
workingday_num    category
weathersit        category
atemp              float64
casual               int64
seasons_autumn    category
seasons_spring    category
seasons_summer    category
seasons_winter    category
dtype: object

In [5]:
# al igual que en la regresión lineal tenemos que separar nuestro dataframe en X e y

X = df.drop("casual", axis = 1)
y = df["casual"]

In [6]:
# y dividir nuestros datos en train y test para poder evaluar la bondad de nuestro modelo

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
# creamos el objeto del modelo, al igual que hacíamos en la regresión lineal
arbol = DecisionTreeRegressor(random_state =0)

# ajustamos el modelo, igual que en la regresión lienal. 
arbol.fit(x_train, y_train)

DecisionTreeRegressor(random_state=0)

In [8]:
# max features. Como vemos, debemos poner en nuestro modelo una profudidad máxima de 4. 

max_features = np.sqrt(len(x_train.columns))
max_features

3.4641016151377544

In [9]:
# max depth

print(arbol.tree_.max_depth)

20


In [10]:
# hacemos las predicciones sobre los dos set de datos el X_test y el X_train
y_pred_test_dt = arbol.predict(x_test)
y_pred_train_dt = arbol.predict(x_train)

In [11]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [12]:
# sacamos las métricas para ver si hay overfitting o unerfitting, para modificar la profundidad en función de estos resultados

dt_results1 = metricas(y_test, y_train, y_pred_test_dt, y_pred_train_dt, "Casual_Cod_Decission Tree I")
dt_results1

,MAE,MSE,RMSE,R2,set,modelo
0,293.136986,252906.438356,502.89804,0.473577,test,Casual_Cod_Decission Tree I
1,0.000000,0.000000,0.00000,1.000000,train,Casual_Cod_Decission Tree I


In [13]:
df.shape

(730, 13)

Overfitting

In [120]:
# lo primero que tenemos que hacer es definir un diccionario con los hiperparámetros que queremos modificar y los valores que queremos 

param = {"max_depth": [4,6,8], # 
        "max_features": [4,5,6,7],# 
        "min_samples_split": [5, 20, 75], 
        "min_samples_leaf": [5,20,75]} 

In [121]:
# una vez creado el diccionario iniciaremos el modelo con GridSearch

gs = GridSearchCV(
            estimator=DecisionTreeRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            verbose=-1, # para que no nos printee ningún mensaje en pantalla
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [122]:
# ajustamos el modelo que acabamos de definir en el GridSearch

gs.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [4, 6, 8], 'max_features': [4, 5, 6, 7],
                         'min_samples_leaf': [5, 20, 75],
                         'min_samples_split': [5, 20, 75]},
             return_train_score=True, scoring='neg_mean_squared_error',
             verbose=-1)

In [123]:
# este método nos esta diciendo que el mejor modelo es aquel que tiene una profundidad de 6, que usa 4 variables predictoras para construir el modelo y que tiene  un min_samples_leaf y un min_samples_split de 10. 
mejor_modelo = gs.best_estimator_
mejor_modelo

DecisionTreeRegressor(max_depth=8, max_features=7, min_samples_leaf=5,
                      min_samples_split=20)

In [124]:
# veamos ahora que pinta tiene nuestro árbol


# fig = plt.figure(figsize=(40, 20))
# tree.plot_tree(mejor_modelo, feature_names=x_train.columns, filled=True);

In [125]:
y_pred_test_dt2 = mejor_modelo.predict(x_test)
y_pred_train_dt2 = mejor_modelo.predict(x_train)

In [126]:
dt_results2 = metricas(y_test, y_train, y_pred_test_dt2, y_pred_train_dt2, "Casual_Cod_Decision tree II")
dt_results2

,MAE,MSE,RMSE,R2,set,modelo
0,229.881487,127755.394487,357.428866,0.734078,test,Casual_Cod_Decision tree II
1,205.138329,101243.064884,318.187154,0.783533,train,Casual_Cod_Decision tree II


In [127]:
df.head(2)

,yr,mnth,dia_anual,holiday_num,weekday_num,workingday_num,weathersit,atemp,casual,seasons_autumn,seasons_spring,seasons_summer,seasons_winter
0,0,1,1,1,0,0,2,18.18125,331,0.0,0.0,0.0,1.0
1,0,1,2,0,1,1,2,17.68695,131,0.0,0.0,0.0,1.0


In [128]:
# vamos  a juntar los dataframes de los resultados de los modelos para poder compararlos mejor

df_decision_results = pd.concat([dt_results1, dt_results2], axis = 0).reset_index(drop=True)
df_decision_results

,MAE,MSE,RMSE,R2,set,modelo
0,293.136986,252906.438356,502.898040,0.473577,test,Casual_Cod_Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Casual_Cod_Decission Tree I
2,229.881487,127755.394487,357.428866,0.734078,test,Casual_Cod_Decision tree II
3,205.138329,101243.064884,318.187154,0.783533,train,Casual_Cod_Decision tree II


In [129]:
df_decision_results.style.background_gradient()

,MAE,MSE,RMSE,R2,set,modelo
0,293.136986,252906.438356,502.898040,0.473577,test,Casual_Cod_Decission Tree I
1,0.000000,0.000000,0.000000,1.000000,train,Casual_Cod_Decission Tree I
2,229.881487,127755.394487,357.428866,0.734078,test,Casual_Cod_Decision tree II
3,205.138329,101243.064884,318.187154,0.783533,train,Casual_Cod_Decision tree II


In [130]:
# ademas vamos a guardar este dataframe en un csv para 

df_decision_results.to_csv("datos/6.casual_resultados_1_cod.csv")